In [17]:
import tensorflow as tf
import numpy as np

In [18]:
for gpu in tf.config.experimental.list_physical_devices('GPU'):
    tf.config.experimental.set_memory_growth(gpu, True)

print(tf.__version__)
print(np.__version__)

2.1.0
1.18.1


In [19]:
pretrained_model_names = ['ade20k', 'cityscapes', 'voc2012']
pretrained_model_name = pretrained_model_names[2]

In [20]:
from pspnet import PSPNet101, PSPNet50
nb_classes_config = {
    'ade20k': 150,
    'cityscapes': 19,
    'voc2012': 21
}
pspnet_layers_config = {
    'ade20k': 'pspnet50',
    'cityscapes': 'pspnet101',
    'voc2012': 'pspnet101'
}
input_shape_config = {
    'ade20k': (473, 473),
    'cityscapes': (713, 713),
    'voc2012': (473, 473)
}
pspnet_config = {
    'pspnet50': PSPNet50,
    'pspnet101': PSPNet101
}
nb_classes = nb_classes_config[pretrained_model_name]
pspnet_layers = pspnet_layers_config[pretrained_model_name]
input_shape = input_shape_config[pretrained_model_name]
PSPNet = pspnet_config[pspnet_layers]

In [21]:
pspnet_model = PSPNet(nb_classes=nb_classes, input_shape=input_shape, weights=f'{pspnet_layers}_{pretrained_model_name}')
# pspnet_model.model.summary()

No Keras model & weights found, import from npy weights.
Building a PSPNet based on ResNet 101 expecting inputs of shape (473, 473) predicting 21 classes
PSP module will interpolate to a final feature map size of (60, 60)
Importing weights from weights\npy\pspnet101_voc2012.npy
input_2
conv1_1_3x3_s2
conv1_1_3x3_s2_bn
activation_58
conv1_2_3x3
conv1_2_3x3_bn
activation_59
conv1_3_3x3
conv1_3_3x3_bn
activation_60
max_pooling2d_1
activation_61
conv2_1_1x1_reduce
conv2_1_1x1_reduce_bn
activation_62
zero_padding2d_16
conv2_1_3x3
conv2_1_3x3_bn
activation_63
conv2_1_1x1_increase
conv2_1_1x1_proj
conv2_1_1x1_increase_bn
conv2_1_1x1_proj_bn
add_16
activation_64
conv2_2_1x1_reduce
conv2_2_1x1_reduce_bn
activation_65
zero_padding2d_17
conv2_2_3x3
conv2_2_3x3_bn
activation_66
conv2_2_1x1_increase
conv2_2_1x1_increase_bn
add_17
activation_67
conv2_3_1x1_reduce
conv2_3_1x1_reduce_bn
activation_68
zero_padding2d_18
conv2_3_3x3
conv2_3_3x3_bn
activation_69
conv2_3_1x1_increase
conv2_3_1x1_increase_b

In [22]:
# They will get saved into `weights/keras` as well
# These cells save in the root directory
with open(f'{pspnet_layers}_{pretrained_model_name}.json', 'w+') as json_file:
    print(pspnet_model.model.to_json(), file=json_file)
    
tf.keras.models.save_model(pspnet_model.model, f'{pspnet_layers}_{pretrained_model_name}.h5')

In [23]:
from layers_builder import Interp
with open(f'{pspnet_layers}_{pretrained_model_name}.json', 'r') as json_model_file:
    model_json = json_model_file.read()
    model = tf.keras.models.model_from_json(model_json, custom_objects={'Interp': Interp})

In [24]:
model.load_weights(f'{pspnet_layers}_{pretrained_model_name}.h5')
